<a href="https://colab.research.google.com/github/gandhimonil9823/CS_GO_Analysis/blob/main/preprocessing/Aggregate_data_matches.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [100]:
import pandas as pd
import numpy as np
import os

### Mount data from Google Drive

In [101]:
#mount data folder from google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Change dir and check contents

In [102]:
cd '/content/drive/Shareddrives/CS_GO_Data/processed_data_v2'

/content/drive/Shareddrives/CS_GO_Data/processed_data_v2


In [103]:
ls

aggregated_data/  de_inferno.png    de_season.png     mm_grenades_demos.csv
cs_cruise.png     de_mikla.png      de_train.png      out-de_cache.png
damage_data.csv   de_mirage.png     de_tulip.png      out-de_cbble.png
de_cache.png      de_new_nuke.png   grenade_data.csv  out-de_dust2.png
de_cbble.png      de_nuke.png       kill_data.csv     out-de_inferno.png
de_coast.png      de_overpass.png   Log_ROC.png       out-de_mirage.png
de_dust2.png      de_royal.png      map_data.csv      out-de_overpass.png
de_empire.png     de_santorini.png  meta_data.csv     out-de_train.png


### 1) load data


In [104]:
%%time
meta_df = pd.read_csv('/content/drive/Shared drives/CS_GO_Data/processed_data_v2/meta_data.csv')
#map_df = pd.read_csv('/content/drive/Shareddrives/CS_GO_Data/processed_data_v2/map_data.csv')
#kills_df = pd.read_csv('/content/drive/Shareddrives/CS_GO_Data/processed_data_v2/kill_data.csv')
#grenades_df = pd.read_csv('/content/drive/Shareddrives/CS_GO_Data/processed_data_v2/grenade_data.csv')
#damage_df = pd.read_csv('/content/drive/Shareddrives/CS_GO_Data/processed_data_v2/damage_data.csv')

CPU times: user 402 ms, sys: 68.4 ms, total: 470 ms
Wall time: 533 ms


In [105]:
#match_meta_df
#file, map,  num_rounds, winner_score, loser_score, winner_team, avg_eq_val, avg_round_length


In [106]:
meta_df.dtypes
#add the following
#bomb plant


file              object
map               object
round              int64
start_seconds    float64
end_seconds      float64
winner_team       object
winner_side       object
round_type        object
ct_eq_val          int64
t_eq_val           int64
dtype: object

In [107]:
meta_df

,file,map,round,start_seconds,end_seconds,winner_team,winner_side,round_type,ct_eq_val,t_eq_val
0,esea_match_13770997.dem,de_overpass,1,94.30782,160.9591,Team 2,Terrorist,PISTOL_ROUND,4300,4250
1,esea_match_13770997.dem,de_overpass,2,160.95910,279.3998,Team 2,Terrorist,ECO,6300,19400
2,esea_match_13770997.dem,de_overpass,3,279.39980,341.0084,Team 2,Terrorist,SEMI_ECO,7650,19250
3,esea_match_13770997.dem,de_overpass,4,341.00840,435.4259,Team 2,Terrorist,NORMAL,24900,23400
4,esea_match_13770997.dem,de_overpass,5,435.42590,484.2398,Team 1,CounterTerrorist,ECO,5400,20550
...,...,...,...,...,...,...,...,...,...,...
340328,esea_match_13829173.dem,de_mirage,18,1647.20600,1694.6550,Team 1,Terrorist,ECO,4500,21950
340329,esea_match_13829173.dem,de_mirage,19,1694.65500,1753.4380,Team 1,Terrorist,NORMAL,16400,26100
340330,esea_match_13829173.dem,de_mirage,20,1753.43800,1803.6580,Team 1,Terrorist,FORCE_BUY,15100,26000
340331,esea_match_13829173.dem,de_mirage,21,1803.65800,1856.0850,Team 1,Terrorist,NORMAL,17550,25500


In [108]:
smaller_meta_df = meta_df[['file','map','round','winner_team','ct_eq_val','t_eq_val']].copy()
smaller_meta_df

,file,map,round,winner_team,ct_eq_val,t_eq_val
0,esea_match_13770997.dem,de_overpass,1,Team 2,4300,4250
1,esea_match_13770997.dem,de_overpass,2,Team 2,6300,19400
2,esea_match_13770997.dem,de_overpass,3,Team 2,7650,19250
3,esea_match_13770997.dem,de_overpass,4,Team 2,24900,23400
4,esea_match_13770997.dem,de_overpass,5,Team 1,5400,20550
...,...,...,...,...,...,...
340328,esea_match_13829173.dem,de_mirage,18,Team 1,4500,21950
340329,esea_match_13829173.dem,de_mirage,19,Team 1,16400,26100
340330,esea_match_13829173.dem,de_mirage,20,Team 1,15100,26000
340331,esea_match_13829173.dem,de_mirage,21,Team 1,17550,25500


In [109]:
smaller_meta_df['round_duration'] = meta_df['end_seconds'] - meta_df['start_seconds']
smaller_meta_df

,file,map,round,winner_team,ct_eq_val,t_eq_val,round_duration
0,esea_match_13770997.dem,de_overpass,1,Team 2,4300,4250,66.65128
1,esea_match_13770997.dem,de_overpass,2,Team 2,6300,19400,118.44070
2,esea_match_13770997.dem,de_overpass,3,Team 2,7650,19250,61.60860
3,esea_match_13770997.dem,de_overpass,4,Team 2,24900,23400,94.41750
4,esea_match_13770997.dem,de_overpass,5,Team 1,5400,20550,48.81390
...,...,...,...,...,...,...,...
340328,esea_match_13829173.dem,de_mirage,18,Team 1,4500,21950,47.44900
340329,esea_match_13829173.dem,de_mirage,19,Team 1,16400,26100,58.78300
340330,esea_match_13829173.dem,de_mirage,20,Team 1,15100,26000,50.22000
340331,esea_match_13829173.dem,de_mirage,21,Team 1,17550,25500,52.42700


In [110]:
#match_rounds = smaller_meta_df[['file','round','map']].groupby(['file','map']).max()
match_rounds = smaller_meta_df[['file','round','map']].groupby(['file']).max()
#match_rounds.reset_index
match_rounds

,round,map
file,,
esea_match_13770997.dem,25,de_overpass
esea_match_13779704.dem,39,de_cache
esea_match_13779770.dem,29,de_mirage
esea_match_13779775.dem,30,de_inferno
esea_match_13779780.dem,29,de_inferno
...,...,...
esea_match_13829155.dem,29,de_cache
esea_match_13829165.dem,24,de_dust2
esea_match_13829169.dem,18,de_mirage


In [111]:
match_avgs = smaller_meta_df[['file','ct_eq_val','t_eq_val','round_duration']].groupby("file").mean()
match_avgs

,ct_eq_val,t_eq_val,round_duration
file,,,
esea_match_13770997.dem,22204.166667,17814.583333,84.747674
esea_match_13779704.dem,21267.948718,21329.487179,97.655995
esea_match_13779770.dem,20307.407407,16892.592593,94.112689
esea_match_13779775.dem,18981.666667,20176.666667,94.452687
esea_match_13779780.dem,17970.689655,18365.517241,102.916448
...,...,...,...
esea_match_13829155.dem,15477.586207,22377.586207,81.956562
esea_match_13829165.dem,16373.913043,20658.695652,74.316426
esea_match_13829169.dem,20725.000000,12825.000000,87.696972


In [112]:
match_avgs['avg_eq_val'] = match_avgs.loc[: , "ct_eq_val":"t_eq_val"].mean(axis=1)
match_avgs

,ct_eq_val,t_eq_val,round_duration,avg_eq_val
file,,,,
esea_match_13770997.dem,22204.166667,17814.583333,84.747674,20009.375000
esea_match_13779704.dem,21267.948718,21329.487179,97.655995,21298.717949
esea_match_13779770.dem,20307.407407,16892.592593,94.112689,18600.000000
esea_match_13779775.dem,18981.666667,20176.666667,94.452687,19579.166667
esea_match_13779780.dem,17970.689655,18365.517241,102.916448,18168.103448
...,...,...,...,...
esea_match_13829155.dem,15477.586207,22377.586207,81.956562,18927.586207
esea_match_13829165.dem,16373.913043,20658.695652,74.316426,18516.304348
esea_match_13829169.dem,20725.000000,12825.000000,87.696972,16775.000000


In [113]:
match_avgs.drop(columns=['ct_eq_val', 't_eq_val'],inplace=True)
match_avgs

,round_duration,avg_eq_val
file,,
esea_match_13770997.dem,84.747674,20009.375000
esea_match_13779704.dem,97.655995,21298.717949
esea_match_13779770.dem,94.112689,18600.000000
esea_match_13779775.dem,94.452687,19579.166667
esea_match_13779780.dem,102.916448,18168.103448
...,...,...
esea_match_13829155.dem,81.956562,18927.586207
esea_match_13829165.dem,74.316426,18516.304348
esea_match_13829169.dem,87.696972,16775.000000


In [114]:
match_wins = smaller_meta_df[['file','winner_team']]
match_wins

,file,winner_team
0,esea_match_13770997.dem,Team 2
1,esea_match_13770997.dem,Team 2
2,esea_match_13770997.dem,Team 2
3,esea_match_13770997.dem,Team 2
4,esea_match_13770997.dem,Team 1
...,...,...
340328,esea_match_13829173.dem,Team 1
340329,esea_match_13829173.dem,Team 1
340330,esea_match_13829173.dem,Team 1
340331,esea_match_13829173.dem,Team 1


In [115]:
match_wins['team_1_wins'] = match_wins['winner_team'].apply(lambda x: 1 if x=='Team 1' else 0)
match_wins

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,file,winner_team,team_1_wins
0,esea_match_13770997.dem,Team 2,0
1,esea_match_13770997.dem,Team 2,0
2,esea_match_13770997.dem,Team 2,0
3,esea_match_13770997.dem,Team 2,0
4,esea_match_13770997.dem,Team 1,1
...,...,...,...
340328,esea_match_13829173.dem,Team 1,1
340329,esea_match_13829173.dem,Team 1,1
340330,esea_match_13829173.dem,Team 1,1
340331,esea_match_13829173.dem,Team 1,1


In [116]:
match_wins = match_wins[['file','team_1_wins']].groupby('file').sum()
match_wins

,team_1_wins
file,
esea_match_13770997.dem,9
esea_match_13779704.dem,18
esea_match_13779770.dem,14
esea_match_13779775.dem,16
esea_match_13779780.dem,15
...,...
esea_match_13829155.dem,16
esea_match_13829165.dem,15
esea_match_13829169.dem,15


In [117]:
match_wins = match_wins.merge(match_rounds, on='file', how='outer')
match_wins

,team_1_wins,round,map
file,,,
esea_match_13770997.dem,9,25,de_overpass
esea_match_13779704.dem,18,39,de_cache
esea_match_13779770.dem,14,29,de_mirage
esea_match_13779775.dem,16,30,de_inferno
esea_match_13779780.dem,15,29,de_inferno
...,...,...,...
esea_match_13829155.dem,16,29,de_cache
esea_match_13829165.dem,15,24,de_dust2
esea_match_13829169.dem,15,18,de_mirage


In [118]:
match_wins['team_2_wins'] = match_wins['round'] - match_wins['team_1_wins']
match_wins

,team_1_wins,round,map,team_2_wins
file,,,,
esea_match_13770997.dem,9,25,de_overpass,16
esea_match_13779704.dem,18,39,de_cache,21
esea_match_13779770.dem,14,29,de_mirage,15
esea_match_13779775.dem,16,30,de_inferno,14
esea_match_13779780.dem,15,29,de_inferno,14
...,...,...,...,...
esea_match_13829155.dem,16,29,de_cache,13
esea_match_13829165.dem,15,24,de_dust2,9
esea_match_13829169.dem,15,18,de_mirage,3


In [119]:
match_wins['team1-team2'] =  match_wins['team_1_wins'] - match_wins['team_2_wins']
match_wins

,team_1_wins,round,map,team_2_wins,team1-team2
file,,,,,
esea_match_13770997.dem,9,25,de_overpass,16,-7
esea_match_13779704.dem,18,39,de_cache,21,-3
esea_match_13779770.dem,14,29,de_mirage,15,-1
esea_match_13779775.dem,16,30,de_inferno,14,2
esea_match_13779780.dem,15,29,de_inferno,14,1
...,...,...,...,...,...
esea_match_13829155.dem,16,29,de_cache,13,3
esea_match_13829165.dem,15,24,de_dust2,9,6
esea_match_13829169.dem,15,18,de_mirage,3,12


In [120]:
def findwinner (x):
  if x >0: return 'Team 1'
  elif x<0: return 'Team 2'
  else: return 'draw'

match_wins['winner'] = match_wins['team1-team2'].apply(lambda x: findwinner(x))
match_wins

,team_1_wins,round,map,team_2_wins,team1-team2,winner
file,,,,,,
esea_match_13770997.dem,9,25,de_overpass,16,-7,Team 2
esea_match_13779704.dem,18,39,de_cache,21,-3,Team 2
esea_match_13779770.dem,14,29,de_mirage,15,-1,Team 2
esea_match_13779775.dem,16,30,de_inferno,14,2,Team 1
esea_match_13779780.dem,15,29,de_inferno,14,1,Team 1
...,...,...,...,...,...,...
esea_match_13829155.dem,16,29,de_cache,13,3,Team 1
esea_match_13829165.dem,15,24,de_dust2,9,6,Team 1
esea_match_13829169.dem,15,18,de_mirage,3,12,Team 1


In [121]:
match_avgs
match = match_wins.merge(match_avgs, on='file', how='outer')
match

,team_1_wins,round,map,team_2_wins,team1-team2,winner,round_duration,avg_eq_val
file,,,,,,,,
esea_match_13770997.dem,9,25,de_overpass,16,-7,Team 2,84.747674,20009.375000
esea_match_13779704.dem,18,39,de_cache,21,-3,Team 2,97.655995,21298.717949
esea_match_13779770.dem,14,29,de_mirage,15,-1,Team 2,94.112689,18600.000000
esea_match_13779775.dem,16,30,de_inferno,14,2,Team 1,94.452687,19579.166667
esea_match_13779780.dem,15,29,de_inferno,14,1,Team 1,102.916448,18168.103448
...,...,...,...,...,...,...,...,...
esea_match_13829155.dem,16,29,de_cache,13,3,Team 1,81.956562,18927.586207
esea_match_13829165.dem,15,24,de_dust2,9,6,Team 1,74.316426,18516.304348
esea_match_13829169.dem,15,18,de_mirage,3,12,Team 1,87.696972,16775.000000


In [122]:
match.rename(columns={"round": "num_rounds", "round_duration": "avg_round_duration", "team_2_wins": "team_2_score", "team_1_wins": "team_1_score", "avg_eq_val": "avg_equip_val"},inplace=True)
match

,team_1_score,num_rounds,map,team_2_score,team1-team2,winner,avg_round_duration,avg_equip_val
file,,,,,,,,
esea_match_13770997.dem,9,25,de_overpass,16,-7,Team 2,84.747674,20009.375000
esea_match_13779704.dem,18,39,de_cache,21,-3,Team 2,97.655995,21298.717949
esea_match_13779770.dem,14,29,de_mirage,15,-1,Team 2,94.112689,18600.000000
esea_match_13779775.dem,16,30,de_inferno,14,2,Team 1,94.452687,19579.166667
esea_match_13779780.dem,15,29,de_inferno,14,1,Team 1,102.916448,18168.103448
...,...,...,...,...,...,...,...,...
esea_match_13829155.dem,16,29,de_cache,13,3,Team 1,81.956562,18927.586207
esea_match_13829165.dem,15,24,de_dust2,9,6,Team 1,74.316426,18516.304348
esea_match_13829169.dem,15,18,de_mirage,3,12,Team 1,87.696972,16775.000000


In [123]:
list(match.columns.values)

['team_1_score',
 'num_rounds',
 'map',
 'team_2_score',
 'team1-team2',
 'winner',
 'avg_round_duration',
 'avg_equip_val']

In [124]:
#reorder columns
match = match[['map', 'num_rounds', 'avg_round_duration', 'avg_equip_val', 'team_1_score', 'team_2_score', 'winner']]
match

,map,num_rounds,avg_round_duration,avg_equip_val,team_1_score,team_2_score,winner
file,,,,,,,
esea_match_13770997.dem,de_overpass,25,84.747674,20009.375000,9,16,Team 2
esea_match_13779704.dem,de_cache,39,97.655995,21298.717949,18,21,Team 2
esea_match_13779770.dem,de_mirage,29,94.112689,18600.000000,14,15,Team 2
esea_match_13779775.dem,de_inferno,30,94.452687,19579.166667,16,14,Team 1
esea_match_13779780.dem,de_inferno,29,102.916448,18168.103448,15,14,Team 1
...,...,...,...,...,...,...,...
esea_match_13829155.dem,de_cache,29,81.956562,18927.586207,16,13,Team 1
esea_match_13829165.dem,de_dust2,24,74.316426,18516.304348,15,9,Team 1
esea_match_13829169.dem,de_mirage,18,87.696972,16775.000000,15,3,Team 1


In [125]:
match['winner'].value_counts()

Team 1    6816
Team 2    6082
draw       563
Name: winner, dtype: int64

In [126]:
#class=1 if Team 1 won
#class=-1 if Team 2 won
#class=0 if draw

def findwinnerclass (x):
  if x=='Team 1': return 1
  elif x=='Team 2': return -1
  else: return 0 #draw

match['winner_class'] = match_wins['winner'].apply(lambda x: findwinnerclass(x))
match

,map,num_rounds,avg_round_duration,avg_equip_val,team_1_score,team_2_score,winner,winner_class
file,,,,,,,,
esea_match_13770997.dem,de_overpass,25,84.747674,20009.375000,9,16,Team 2,-1
esea_match_13779704.dem,de_cache,39,97.655995,21298.717949,18,21,Team 2,-1
esea_match_13779770.dem,de_mirage,29,94.112689,18600.000000,14,15,Team 2,-1
esea_match_13779775.dem,de_inferno,30,94.452687,19579.166667,16,14,Team 1,1
esea_match_13779780.dem,de_inferno,29,102.916448,18168.103448,15,14,Team 1,1
...,...,...,...,...,...,...,...,...
esea_match_13829155.dem,de_cache,29,81.956562,18927.586207,16,13,Team 1,1
esea_match_13829165.dem,de_dust2,24,74.316426,18516.304348,15,9,Team 1,1
esea_match_13829169.dem,de_mirage,18,87.696972,16775.000000,15,3,Team 1,1


In [127]:
def create_map_cols (df):
  unique_maps = df["map"].unique()

  map_dataframe = pd.DataFrame()

  for map_cs_go in unique_maps:
    map_dataframe[map_cs_go] = df["map"].apply(lambda x: 1 if x == map_cs_go else 0)

  df = df.merge(map_dataframe, on='file', how='inner')
  df.drop(columns='map',inplace=True)
  
  return df


In [128]:
match = create_map_cols(match)
match

,num_rounds,avg_round_duration,avg_equip_val,team_1_score,team_2_score,winner,winner_class,de_overpass,de_cache,de_mirage,de_inferno,de_train,de_dust2,de_cbble,de_nuke
file,,,,,,,,,,,,,,,
esea_match_13770997.dem,25,84.747674,20009.375000,9,16,Team 2,-1,1,0,0,0,0,0,0,0
esea_match_13779704.dem,39,97.655995,21298.717949,18,21,Team 2,-1,0,1,0,0,0,0,0,0
esea_match_13779770.dem,29,94.112689,18600.000000,14,15,Team 2,-1,0,0,1,0,0,0,0,0
esea_match_13779775.dem,30,94.452687,19579.166667,16,14,Team 1,1,0,0,0,1,0,0,0,0
esea_match_13779780.dem,29,102.916448,18168.103448,15,14,Team 1,1,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
esea_match_13829155.dem,29,81.956562,18927.586207,16,13,Team 1,1,0,1,0,0,0,0,0,0
esea_match_13829165.dem,24,74.316426,18516.304348,15,9,Team 1,1,0,0,0,0,0,1,0,0
esea_match_13829169.dem,18,87.696972,16775.000000,15,3,Team 1,1,0,0,1,0,0,0,0,0


In [129]:
%%time
match.to_csv('/content/drive/Shareddrives/CS_GO_Data/processed_data_v2/aggregated_data/matches_agg.csv')

CPU times: user 85.4 ms, sys: 971 µs, total: 86.4 ms
Wall time: 107 ms


In [130]:
match_agg_predictions = match.copy()
match_agg_predictions = match_agg_predictions.reset_index()
match_agg_predictions

,file,num_rounds,avg_round_duration,avg_equip_val,team_1_score,team_2_score,winner,winner_class,de_overpass,de_cache,de_mirage,de_inferno,de_train,de_dust2,de_cbble,de_nuke
0,esea_match_13770997.dem,25,84.747674,20009.375000,9,16,Team 2,-1,1,0,0,0,0,0,0,0
1,esea_match_13779704.dem,39,97.655995,21298.717949,18,21,Team 2,-1,0,1,0,0,0,0,0,0
2,esea_match_13779770.dem,29,94.112689,18600.000000,14,15,Team 2,-1,0,0,1,0,0,0,0,0
3,esea_match_13779775.dem,30,94.452687,19579.166667,16,14,Team 1,1,0,0,0,1,0,0,0,0
4,esea_match_13779780.dem,29,102.916448,18168.103448,15,14,Team 1,1,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13456,esea_match_13829155.dem,29,81.956562,18927.586207,16,13,Team 1,1,0,1,0,0,0,0,0,0
13457,esea_match_13829165.dem,24,74.316426,18516.304348,15,9,Team 1,1,0,0,0,0,0,1,0,0
13458,esea_match_13829169.dem,18,87.696972,16775.000000,15,3,Team 1,1,0,0,1,0,0,0,0,0
13459,esea_match_13829171.dem,24,75.069213,18711.458333,16,8,Team 1,1,0,0,0,0,1,0,0,0


In [131]:
match_agg_predictions.columns

Index(['file', 'num_rounds', 'avg_round_duration', 'avg_equip_val',
       'team_1_score', 'team_2_score', 'winner', 'winner_class', 'de_overpass',
       'de_cache', 'de_mirage', 'de_inferno', 'de_train', 'de_dust2',
       'de_cbble', 'de_nuke'],
      dtype='object')

In [132]:
match_agg_predictions.drop(columns=['file','team_1_score','team_2_score','winner'],inplace=True)
match_agg_predictions = match_agg_predictions[['num_rounds', 'avg_round_duration', 'avg_equip_val',
       'de_overpass', 'de_cache', 'de_mirage', 'de_inferno', 'de_train',
       'de_dust2', 'de_cbble', 'de_nuke', 'winner_class']] #reorder cols


In [133]:
match_agg_predictions

,num_rounds,avg_round_duration,avg_equip_val,de_overpass,de_cache,de_mirage,de_inferno,de_train,de_dust2,de_cbble,de_nuke,winner_class
0,25,84.747674,20009.375000,1,0,0,0,0,0,0,0,-1
1,39,97.655995,21298.717949,0,1,0,0,0,0,0,0,-1
2,29,94.112689,18600.000000,0,0,1,0,0,0,0,0,-1
3,30,94.452687,19579.166667,0,0,0,1,0,0,0,0,1
4,29,102.916448,18168.103448,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
13456,29,81.956562,18927.586207,0,1,0,0,0,0,0,0,1
13457,24,74.316426,18516.304348,0,0,0,0,0,1,0,0,1
13458,18,87.696972,16775.000000,0,0,1,0,0,0,0,0,1
13459,24,75.069213,18711.458333,0,0,0,0,1,0,0,0,1


In [134]:
%%time
match_agg_predictions.to_csv('/content/drive/Shareddrives/CS_GO_Data/processed_data_v2/aggregated_data/modelinput/winner_matches_agg.csv',index=False)

CPU times: user 81.6 ms, sys: 1.84 ms, total: 83.5 ms
Wall time: 98.1 ms


In [135]:
test_load = pd.read_csv('/content/drive/Shareddrives/CS_GO_Data/processed_data_v2/aggregated_data/modelinput/winner_matches_agg.csv')
test_load

,num_rounds,avg_round_duration,avg_equip_val,de_overpass,de_cache,de_mirage,de_inferno,de_train,de_dust2,de_cbble,de_nuke,winner_class
0,25,84.747674,20009.375000,1,0,0,0,0,0,0,0,-1
1,39,97.655995,21298.717949,0,1,0,0,0,0,0,0,-1
2,29,94.112689,18600.000000,0,0,1,0,0,0,0,0,-1
3,30,94.452687,19579.166667,0,0,0,1,0,0,0,0,1
4,29,102.916448,18168.103448,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
13456,29,81.956562,18927.586207,0,1,0,0,0,0,0,0,1
13457,24,74.316426,18516.304348,0,0,0,0,0,1,0,0,1
13458,18,87.696972,16775.000000,0,0,1,0,0,0,0,0,1
13459,24,75.069213,18711.458333,0,0,0,0,1,0,0,0,1
